# Neuroevoluce

Pomocí evolučních algoritmů můžeme vyvíjet i celé neuronové sítě. Buď můžeme evolvovat váhy mezi neurony, strukturu sítě nebo obojí najednou. Nejprve se podíváme, jak naimplementovat jednoduchý algoritmus pro evoluci vah, následně si zkusíme i evoluci vah a struktury sítě najednou pomocí algoritmu NEAT.

## Evoluce vah

Evoluce vah je jednoduchá, stačí naimplementovat jednoduchou neuronovou síť, které umíme nastavit parametry. Náš jedinec v evoluci bude jeden dlouhý vektor, který bude reprezentovat váhy mezi vrstvami v neuronové síti. Při vyhodnocování tedy nastavíme síti váhy podle daného jedince a spočítáme její výstup. Zkusíme si tímto způsobem implementovat klasifikaci na našem oblíbeném datasetu iris.

In [1]:
from sklearn import datasets, model_selection
import numpy as np
import collections
from deap import algorithms, creator, base, benchmarks, cma, tools

Začneme tím, že si připravíme jednoduchou implementaci neuronové sítě. Při inicializaci nastavíme vrstvy, jejich velikosti a také jejich aktivace, jelikož každá vrstva může mít svou vlastní aktivační funkci. Dále budeme potřebovat jednoho dlouhého linárního jedince všech vah rozdělit na váhové matice. To uděláme tak, že budeme brát vždy velikost akutální vrstvy a velikost té následující a na základě toho vezmeme odpovídající část jedince a z něj vytvoříme matici vah. Zároveň musíme přidat biasy, takže velikost aktuální vrstvy zvětšíme ještě o 1. Nakonec bude potřeba ještě funkce pro evaluaci sítě, která vytvořenou síť vyhodnotí tak, že pro každou vrstvu použije odpovídající aktivaci a vrátí nám výstup sítě.

In [2]:
class NeuralNetwork: 
    def __init__(self, layer_sizes, activations):
        self.layer_sizes = layer_sizes
        self.layers = None

        # kontrola, zda pocet akivaci sedi na pocet vrstev
        if len(list(activations)) != len(layer_sizes) - 1:
            raise AttributeError("Number of activations does not match number of layers")
        
        self.activations = list(activations)
        self.vectorized_net = None

    # vraci velikosti vahovych matic mezi vrstvami
    def vectorized_size(self):
        return sum(map(lambda x: (x[0] + 1) * x[1], zip(self.layer_sizes, self.layer_sizes[1:])))

    # vezme sit a nastavi hodnoty vahovych matic podle jedince 
    def set_weights(self, vectorized_net):
        
        # kontrola, zda si velikosti odpovidaji
        if len(vectorized_net) != self.vectorized_size():
            raise AttributeError(
                f"Length of vector does not match vectorized_size: {len(vectorized_net)} != {self.vectorized_size()}")

        self.vectorized_net = vectorized_net
        self.layers = []
        sum_sizes = 0
        
        # prochazim vrstvy a beru vzdy aktualni a 1 nasledujici a vyrabim vahove matice tim ze vybiram spravnou cast jedince
        for (p, n) in zip(self.layer_sizes, self.layer_sizes[1:]):
            layer = vectorized_net[sum_sizes: sum_sizes + (p + 1) * n]
            self.layers.append(np.reshape(layer, newshape=(p + 1, n))) # reshape do spravneho tvaru
            sum_sizes += (p + 1) * n # soucet velikosti predchozich vrstev pro indexaci


    # vyhodnoceni site
    def eval_network(self, inputs):
        activations = inputs
        
        for act_func, layer in zip(self.activations, self.layers):
            activations_1 = np.append(np.array([1.0]), activations)  # rozsireni matice vah o konstantu 1.0 pro biasy
            activations = act_func(np.dot(activations_1, layer))

        return activations

Nyní si akorát načteme data a rozdělíme si je na trénovací a testovací.

In [3]:
iris = datasets.load_iris()
train_x, test_x, train_y, test_y = model_selection.train_test_split(iris.data, iris.target)

Musíme si ještě definovat fitness funkci, což bude pouze vyhodnocení sítě na datech, kdy vždy vybereme predikovanou třídu s maximální pravděpodobností a spočítáme pak celkovou accuracy. Zároveň si definujeme vlastní lineární a relu aktivace.

In [4]:
def relu(x):
    return np.maximum(0,x)

def linear(x):
    return x

def fitness(ind, X, y):
    net.set_weights(ind)
    
    acc = 0
    for xi, yi in zip(X, y):
        if np.argmax(net.eval_network(xi)) == yi:
            acc += 1
    
    return acc/len(y),

Pro samotné nastavení vah neuronové sítě použíjeme algoritmus CMA-ES, což je evoluční strategie, která je dobrá na úlohy spojité optimalizace, kterou máme i zde. V evoluční strategii se nevyskytuje křížení, ale místo něj se klade důraz na mutaci. Zde se používá námi známá gaussovská mutace. Evoluční strategii používáme také proto, že nám umožňuje váhy měnit korelovaně, protože si pamatuje celou kovariační matici, ze které generuje jedince. Klasická evoluce umí měnit váhy jen nezávisle na sobě, a dává proto horší výsledky. CMA-ES je implementována v knihovně deap, a je tedy snadné ji použít.

In [5]:
net = NeuralNetwork([4, 5, 3], [relu, linear])
ind_size = net.vectorized_size()    
    
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("evaluate", fitness, X=train_x, y=train_y)

strategy = cma.Strategy(centroid=[0.0]*ind_size, sigma=0.1, lambda_=5*ind_size)
toolbox.register("generate", strategy.generate, creator.Individual)
toolbox.register("update", strategy.update)

hof = tools.HallOfFame(1)
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)

_ = algorithms.eaGenerateUpdate(toolbox, ngen=250, stats=stats, halloffame=hof)

gen	nevals	avg     	std      	min	max     
0  	215   	0.331852	0.0819991	0  	0.669643
1  	215   	0.330565	0.114867 	0  	0.669643
2  	215   	0.334759	0.0916622	0  	0.660714
3  	215   	0.329444	0.0823641	0  	0.669643
4  	215   	0.333306	0.102631 	0  	0.848214
5  	215   	0.340947	0.0806923	0  	0.669643
6  	215   	0.341279	0.0694954	0.0535714	0.732143
7  	215   	0.354527	0.072225 	0.133929 	0.767857
8  	215   	0.344145	0.0842591	0        	0.669643
9  	215   	0.355814	0.083147 	0.00892857	0.669643
10 	215   	0.367442	0.101597 	0         	0.732143
11 	215   	0.387417	0.134757 	0         	0.741071
12 	215   	0.395017	0.125434 	0         	0.839286
13 	215   	0.414286	0.140467 	0         	0.75    
14 	215   	0.42338 	0.142961 	0         	0.794643
15 	215   	0.418189	0.146397 	0.00892857	0.785714
16 	215   	0.421138	0.15088  	0.00892857	0.964286
17 	215   	0.435797	0.15275  	0         	0.875   
18 	215   	0.459718	0.159689 	0         	0.964286
19 	215   	0.451038	0.156965 	0.116071  	0.946429
20

163	215   	0.972716	0.0026772 	0.955357  	0.973214
164	215   	0.972924	0.00217617	0.946429  	0.973214
165	215   	0.973173	0.000607506	0.964286  	0.973214
166	215   	0.973007	0.00181493 	0.955357  	0.973214
167	215   	0.973007	0.00134567 	0.964286  	0.973214
168	215   	0.97309 	0.0010473  	0.964286  	0.973214
169	215   	0.973131	0.000857133	0.964286  	0.973214
170	215   	0.97309 	0.00135588 	0.955357  	0.973214
171	215   	0.972965	0.00170418 	0.955357  	0.973214
172	215   	0.973131	0.00121501 	0.955357  	0.973214
173	215   	0.973048	0.00120647 	0.964286  	0.973214
174	215   	0.97309 	0.0010473  	0.964286  	0.973214
175	215   	0.973048	0.00148227 	0.955357  	0.973214
176	215   	0.973007	0.00181493 	0.955357  	0.973214
177	215   	0.973007	0.00159762 	0.955357  	0.973214
178	215   	0.972924	0.00217617 	0.955357  	0.973214
179	215   	0.972841	0.00216346 	0.955357  	0.973214
180	215   	0.972799	0.00254985 	0.955357  	0.973214
181	215   	0.972882	0.00168995 	0.964286  	0.973214
182	215   	0.9

Na závěr si zkusíme otestovat síť na testovacích datech a změřit na nich accuracy.

In [7]:
fitness(hof[0], test_x, test_y)

(1.0,)

## NEAT

Zkusíme se podívat na algoritmus NEAT na příkladu s datasetem iris a zkusíme pro něj najít nejvhodnější síť, která bude umět data nejlépe klasifikovat. Samotná implementace algorimtu není vůbec jednoduchá, ale naštěstí v pythonu existuje knihovna neat-python, kde už tu práci někdo udělal za nás. Knihovna má trochu jiný interface, než na co jsme zvyklí, ale není složité se jí naučit používat. Nastavení parametrů algoritmu se načítá ze zvláštního souboru *config-feedforward*. Dokumentace k jeho formátování se nachází [zde](https://neat-python.readthedocs.io/en/latest/config_file.html). Potom už stačí jen algoritmus spustit. Níže uvedený příklad vychází z dokumentace k [neat-python](https://github.com/CodeReclaimers/neat-python/tree/master/examples/xor) upravený pro naši klasifikaci irisů.

In [10]:
import neat
import visualize
import os
#os.environ["PATH"] += os.pathsep + 'C:/Users/katie/Anaconda3/Library/bin/graphviz'

# vytvoreni a vyhodnoceni site v neatu - fitness funkce
def eval_genomes(genomes, config):
    for genome_id, genome in genomes:
        genome.fitness = 0.0
        net = neat.nn.FeedForwardNetwork.create(genome, config) 
        for xi, xo in zip(train_x, train_y): 
            output = net.activate(xi)
            if np.argmax(output) == xo:
                genome.fitness += 1

def run(config_file):
    
    # nacteme konfiguraci neatu 
    config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                         neat.DefaultSpeciesSet, neat.DefaultStagnation,
                         config_file)
    # vytvorime novou populaci
    p = neat.Population(config)

    # pridame statisticke vypocty pro logovani evoluce    
    p.add_reporter(neat.StdOutReporter(show_species_detail=True))
    stats = neat.StatisticsReporter()
    p.add_reporter(stats)
    
    # pustime pro dany pocet generaci
    winner = p.run(eval_genomes, 100)

    # zobrazime nejlepsiho jedince a jeho vystup
    print('\nBest genome:\n{!s}'.format(winner))
    print('\nOutput:')
    winner_net = neat.nn.FeedForwardNetwork.create(winner, config)
    for xi, xo in zip(train_x, train_y):
        output = winner_net.activate(xi)
        print("input {!r}, expected output {!r}, got {!r}".format(xi, xo, output))

    # vykresluje nejlepsi sit
    node_names = {-1:'x1', -2: 'x2', -3: 'x3', -4: 'x4', 0:'setosa', 1:'virginica', 2:'versicolor'}
    visualize.draw_net(config, winner, True, node_names=node_names) 
    visualize.plot_stats(stats, ylog=False, view=True)
    visualize.plot_species(stats, view=True)

    #p = neat.Checkpointer.restore_checkpoint('neat-checkpoint-4')
    #p.run(eval_genomes, 10)

if __name__ == '__main__':
    # urceni cesty ke konfiguracnimu souboru
    config_path = os.path.join('.', 'config-feedforward')
    run(config_path)


 ****** Running generation 0 ****** 

Population's average fitness: 36.10667 stdev: 13.89443
Best fitness: 85.00000 - size: (3, 12) - species 1 - id 103
Average adjusted fitness: 0.425
Mean genetic distance 1.113, standard deviation 0.274
Population of 150 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    0   150     85.0    0.425     0
Total extinctions: 0
Generation time: 0.829 sec

 ****** Running generation 1 ****** 

Population's average fitness: 42.40000 stdev: 15.27438
Best fitness: 85.00000 - size: (3, 12) - species 1 - id 103
Average adjusted fitness: 0.499
Mean genetic distance 1.249, standard deviation 0.280
Population of 150 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    1   150     85.0    0.499     1
Total extinctions: 0
Generation time: 0.742 sec (0.786 average)

 ****** Running generation 2 ****** 

Population's average fitness: 43.4866

Population's average fitness: 56.22667 stdev: 18.24359
Best fitness: 102.00000 - size: (4, 14) - species 1 - id 1104
Average adjusted fitness: 0.421
Mean genetic distance 1.658, standard deviation 0.292
Population of 150 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1   18   150    102.0    0.421    11
Total extinctions: 0
Generation time: 1.318 sec (1.013 average)

 ****** Running generation 19 ****** 

Population's average fitness: 54.92667 stdev: 17.62842
Best fitness: 102.00000 - size: (4, 14) - species 1 - id 1104
Average adjusted fitness: 0.287
Mean genetic distance 1.611, standard deviation 0.354
Population of 150 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1   19   150    102.0    0.287    12
Total extinctions: 0
Generation time: 1.072 sec (1.036 average)

 ****** Running generation 20 ****** 

Population's average fitness: 55.18667 stdev: 18.35770

Population's average fitness: 55.44667 stdev: 18.90839
Best fitness: 106.00000 - size: (3, 7) - species 1 - id 4122
Average adjusted fitness: 0.253
Mean genetic distance 2.282, standard deviation 0.680
Population of 149 members in 3 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1   35    47    106.0    0.266     8
     2   13    65    102.0    0.273    10
     3    1    37     75.0    0.220     0
Total extinctions: 0
Generation time: 0.699 sec (0.709 average)

 ****** Running generation 36 ****** 

Population's average fitness: 56.49664 stdev: 19.22374
Best fitness: 106.00000 - size: (3, 7) - species 1 - id 4122
Average adjusted fitness: 0.323
Mean genetic distance 2.145, standard deviation 0.601
Population of 150 members in 3 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1   36    48    106.0    0.315     9
     2   14    66    102.0    0.367    11
     3    2    36     75.0    0.28

Population's average fitness: 56.53642 stdev: 19.19567
Best fitness: 108.00000 - size: (3, 6) - species 1 - id 6432
Average adjusted fitness: 0.367
Mean genetic distance 1.910, standard deviation 0.549
Population of 150 members in 3 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1   50    57    108.0    0.332     7
     2   28    47     75.0    0.337     5
     3   16    46    107.0    0.432     1
Total extinctions: 0
Generation time: 0.774 sec (0.702 average)

 ****** Running generation 51 ****** 

Population's average fitness: 58.58000 stdev: 19.35055
Best fitness: 108.00000 - size: (3, 6) - species 1 - id 6432
Average adjusted fitness: 0.305
Mean genetic distance 1.903, standard deviation 0.563
Population of 151 members in 3 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1   51    49    108.0    0.288     8
     2   29    51     75.0    0.287     6
     3   17    51    107.0    0.34

Population's average fitness: 63.22000 stdev: 20.21876
Best fitness: 108.00000 - size: (3, 5) - species 2 - id 9506
Average adjusted fitness: 0.461
Mean genetic distance 2.048, standard deviation 0.540
Population of 150 members in 3 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1   65    52    108.0    0.424    22
     2   43    51    108.0    0.486     0
     3   31    47    107.0    0.474    16
Total extinctions: 0
Generation time: 0.729 sec (0.733 average)

 ****** Running generation 66 ****** 

Population's average fitness: 60.75333 stdev: 20.47240
Best fitness: 108.00000 - size: (3, 5) - species 2 - id 9506
Average adjusted fitness: 0.337
Mean genetic distance 2.062, standard deviation 0.504
Population of 150 members in 3 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1   66    52    108.0    0.270    23
     2   44    44    108.0    0.316     1
     3   32    54    107.0    0.42

Population's average fitness: 60.88742 stdev: 20.27380
Best fitness: 108.00000 - size: (3, 5) - species 2 - id 11299
Average adjusted fitness: 0.331
Mean genetic distance 2.100, standard deviation 0.619
Population of 150 members in 3 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1   81    59    108.0    0.339    38
     2   59    59    108.0    0.350    16
     4    7    32     75.0    0.304     4
Total extinctions: 0
Generation time: 0.861 sec (0.736 average)

 ****** Running generation 82 ****** 

Population's average fitness: 56.18667 stdev: 21.23343
Best fitness: 108.00000 - size: (3, 5) - species 2 - id 11299
Average adjusted fitness: 0.502
Mean genetic distance 2.178, standard deviation 0.623
Population of 150 members in 3 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1   82    54    108.0    0.501    39
     2   60    57    108.0    0.503    17
     4    8    39     75.0    0.

Population's average fitness: 61.99338 stdev: 20.49083
Best fitness: 108.00000 - size: (3, 5) - species 2 - id 11299
Average adjusted fitness: 0.375
Mean genetic distance 2.096, standard deviation 0.640
Population of 150 members in 3 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1   96    47    108.0    0.341    53
     2   74    55    108.0    0.437    31
     4   22    48     75.0    0.346    19
Total extinctions: 0
Generation time: 0.673 sec (0.704 average)

 ****** Running generation 97 ****** 

Population's average fitness: 61.95333 stdev: 21.88221
Best fitness: 108.00000 - size: (3, 5) - species 2 - id 11299

Species 4 with 48 members is stagnated: removing it
Average adjusted fitness: 0.397
Mean genetic distance 1.838, standard deviation 0.655
Population of 150 members in 2 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1   97    67    108.0    0.334    54
     2   75    83    

ExecutableNotFound: failed to execute ['dot', '-Tsvg', '-O', 'Digraph.gv'], make sure the Graphviz executables are on your systems' PATH

## Úkol na cvičení

Zkuste si pohrát s NEATem a vyskoušet si ho třeba na OpenAI Gym, který jsme dělali na druhém cvičení. Bude potřeba upravit  fitness funkci, tedy místo accuracy dát odměnu za hru a také bude potřeba změnit pár věcí v konfiguračním souborum například parametry počáteční sítě a podmínky, kdy má algoritmus skončit. [Zde](https://github.com/CodeReclaimers/neat-python/tree/master/examples/openai-lander) je pro inspiraci vyřešený [LunarLander](https://gym.openai.com/envs/LunarLander-v2/). Můžete si vybrat libovolný z [problémů](https://gym.openai.com/envs/#classic_control) z knihovny gym a zkusit vytvořit a natrénovat neuronovou síť pomocí neuroevoluce. 
